In [1]:
# Install Requirments (Updated on 12/17/2024)
# !pip3 install -r requirements.txt

In [2]:
import os
import sys

# Suppress TensorFlow logs
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Use asynchronous CUDA memory allocator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Restrict TensorFlow to GPU 0
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Prevent CUDA errors

# Redirect STDERR to /dev/null to silence C++ warnings
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import tensorflow as tf
import logging

# Suppress TensorFlow and absl logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Clear previous sessions
tf.keras.backend.clear_session()

# GPU configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is configured properly.")
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

sys.stderr = stderr  # Restore STDERR

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


E0000 00:00:1735262566.218507  173723 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735262566.221904  173723 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


No GPU detected. TensorFlow will use the CPU.
Num GPUs Available: 0


In [3]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [17]:
f.predictions('lunr')

LUNR 5m Interval Timestamp: 2024-12-27 20:19:02


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,down,static,static,static
kelly_1:2.5,0.285872,0.14963,0.279159,0.159532,0.2832
prob_up,0.280268,0.221323,0.230697,0.25,0.0
prob_static,0.41482,0.300549,0.386303,0.375,1.0
prob_down,0.304913,0.478128,0.383001,0.375,0.0
precision,0.489908,0.392593,0.485114,0.399666,0.488
recall,0.579176,0.344156,0.600868,0.518438,0.396963
f1,0.530815,0.366782,0.536822,0.451369,0.437799
support,"[463.0, 461.0, 462.0]","[463.0, 461.0, 462.0]","[463.0, 461.0, 462.0]","[463.0, 461.0, 462.0]","[463.0, 461.0, 462.0]"


LUNR 15m Interval Timestamp: 2024-12-27 20:19:26


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,static
kelly_1:2.5,0.23,0.204938,0.239269,0.136893,0.269565
prob_up,0.28108,0.179317,0.259826,0.125,0.0
prob_static,0.499189,0.514329,0.430053,0.5,1.0
prob_down,0.219731,0.306354,0.310121,0.375,0.0
precision,0.45,0.432099,0.456621,0.383495,0.478261
recall,0.50625,0.4375,0.625,0.49375,0.34375
f1,0.476471,0.434783,0.527704,0.431694,0.4
support,"[161.0, 160.0, 160.0]","[161.0, 160.0, 160.0]","[161.0, 160.0, 160.0]","[161.0, 160.0, 160.0]","[161.0, 160.0, 160.0]"


LUNR 1h Interval Timestamp: 2024-12-27 20:19:41


...

LUNR 1d Interval Timestamp: 2024-12-27 20:20:15


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,down,down,down,down,up
kelly_1:2.5,0.338182,0.326923,0.353846,0.335,0.076
prob_up,0.144679,0.033245,0.272843,0.25,1.0
prob_static,0.010277,0.007167,0.038173,0.0,0.0
prob_down,0.845044,0.959588,0.688984,0.75,0.0
precision,0.527273,0.519231,0.538462,0.525,0.34
recall,0.54717,0.509434,0.660377,0.396226,1.0
f1,0.537037,0.514286,0.59322,0.451613,0.507463
support,"[51.0, 48.0, 53.0]","[51.0, 48.0, 53.0]","[51.0, 48.0, 53.0]","[51.0, 48.0, 53.0]","[51.0, 48.0, 53.0]"


LUNR 1wk Interval Timestamp: 2024-12-27 20:20:28


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,up,up,up
kelly_1:2.5,0.16,0.125,0.125,0.222222,0.15
prob_up,0.975179,0.999432,0.700868,0.75,1.0
prob_static,0.005108,0.00006,0.060561,0.0,0.0
prob_down,0.019713,0.000507,0.23857,0.25,0.0
precision,0.4,0.375,0.375,0.444444,0.392857
recall,0.363636,0.272727,0.272727,0.363636,1.0
f1,0.380952,0.315789,0.315789,0.4,0.564103
support,"[11.0, 7.0, 10.0]","[11.0, 7.0, 10.0]","[11.0, 7.0, 10.0]","[11.0, 7.0, 10.0]","[11.0, 7.0, 10.0]"


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got [1 2]

In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')